In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
import os
sys.path.append(os.path.abspath(".."))

import polars as pl
import pandas as pd
import numpy as np

import backtrader as bt

from shared.poly_utils import get_markets
from backtrader_plotting import Bokeh
from backtrader.feeds import PandasData

from backtrader_plotting.schemes import Blackly
from bokeh.plotting import output_file, save

%load_ext autoreload
%autoreload 2


In [ ]:
# Updated to scan parquet files from ../processed/trades (relative to analysis/)
df = pl.scan_parquet("../processed/trades/**/*.parquet").collect(streaming=True)

# timestamp is already datetime in parquet usually, but ensuring it
df = df.with_columns(
    pl.col("timestamp").cast(pl.Datetime).alias("timestamp")
)

In [ ]:
# Load markets from ../markets_partitioned
markets = get_markets("../markets_partitioned")

In [ ]:
markets = markets.with_columns(
    pl.col("createdAt").str.to_datetime().alias("createdAt")
)

In [ ]:
markets = markets.with_columns(
    pl.col('createdAt').dt.replace_time_zone(None)
)

In [ ]:
target_id = markets.filter(markets['question'].str.contains('Will Donald Trump win the 2024 US Presidential Election')).sort('volume').row(0, named=True)['id']

In [ ]:
sel_df = df.filter(pl.col('market_id') == target_id)

In [ ]:
sel_df

In [ ]:
sel_df = sel_df[['timestamp', 'price', 'usd_amount', 'nonusdc_side']]

In [ ]:
sel_df = sel_df.to_pandas()

In [ ]:
sel_df['price'] = np.where(sel_df['nonusdc_side'] == 'token2', 1 - sel_df['price'], sel_df['price']) #standardize it for ease

In [ ]:
sel_df = sel_df[['timestamp', 'price', 'usd_amount']]

In [ ]:
sel_df = sel_df.set_index('timestamp')

# Resample to 1-minute bars
ohlcv = sel_df.resample('10min').agg({
    'price': ['first', 'max', 'min', 'last'],
    'usd_amount': 'sum'
})

# Flatten column names
ohlcv.columns = ['open', 'high', 'low', 'close', 'volume']

# Reset index if you want timestamp as a column
ohlcv = ohlcv.reset_index()


In [ ]:
ohlcv = ohlcv.dropna()


In [ ]:
ohlcv = ohlcv[ohlcv['timestamp'] >= "2024-10-01"]

In [ ]:
ohlcv

In [ ]:
ohlcv['close'].plot() #sanity check

In [ ]:
ohlcv = ohlcv[ohlcv['timestamp'] != '2024-10-24 21:00:00'] #remove broken data for aesthetic

In [ ]:
class PandasDataFeed(PandasData):
    params = (
        ('datetime', 'timestamp'),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),  # Set to -1 to disable
    )

In [ ]:
class SimpleMAStrategy(bt.Strategy):
    params = (
        ('fast_period', 50),
        ('slow_period', 200),
        ('stop_loss', 0.02),  # 2% stop loss
        ('trailing_stop', False),  # Enable trailing stop
        ('risk_percent', 0.95),  # Use 95% of portfolio
    )
    
    def __init__(self):
        self.fast_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.fast_period
        )
        self.slow_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.slow_period
        )
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)
        self.crossover.plotinfo.plot = False
        self.crossover.plotinfo.plotmaster = self.data
        
        self.order = None
        self.buy_price = None
    
    def next(self):
        # Check stop loss first if we have a position
        if self.position:
            # Calculate stop loss price
            stop_price = self.buy_price * (1 - self.params.stop_loss)
            
            # Exit on stop loss
            if self.data.close[0] <= stop_price:
                self.close()
                return
            
            # Exit on MA crossover
            if self.crossover < 0:
                self.close()
                return
        
        # Entry logic
        if not self.position:
            if self.crossover > 0:
                # Calculate position size
                cash = self.broker.getcash() * self.params.risk_percent
                size = cash / self.data.close[0]
                self.buy(size=size)
                self.buy_price = self.data.close[0]
    
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price

In [ ]:
# Create cerebro engine
cerebro = bt.Cerebro()

# Prepare data - ensure timestamp is datetime and set as index
ohlcv_bt = ohlcv.copy()
ohlcv_bt['timestamp'] = pd.to_datetime(ohlcv_bt['timestamp'])  # Convert to datetime
ohlcv_bt = ohlcv_bt.set_index('timestamp')  # Set as index

# Add data feed using built-in PandasData
data = bt.feeds.PandasData(
    dataname=ohlcv_bt,
    openinterest=-1
)
cerebro.adddata(data, name='Will Donald Trump win the 2024 US Presidential Election')

# Add strategy
cerebro.addstrategy(SimpleMAStrategy)

# Set initial cash
cerebro.broker.setcash(10000.0)

# Set commission (0.1%)
cerebro.broker.setcommission(commission=0)

# Print starting conditions
print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')

# Run backtest
cerebro.run()

# Print final results
print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')

In [ ]:
scheme = Blackly()



b = Bokeh(
    style='bar', 
    plot_mode='single', 
    scheme=scheme,
    output_mode='save',
    filename='../processed/plot.html' # Save to processed, assuming write access
)
cerebro.plot(b)
